<a href="https://colab.research.google.com/github/Keenandrea/tensorflow2.0/blob/master/LSTMTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0-beta0
import tensorflow as tf

import numpy as np
import os
import time

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
text = open('metakaf.txt', 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 121108 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

One morning, when Gregor Samsa woke from troubled dreams, he found
himself transformed in his bed into a horrible vermin.  He lay on
his armour-like back, and if he lifted his head a little he could
see his brown belly, slightly domed and divided 


In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  ':' :  11,
  ';' :  12,
  '?' :  13,
  'A' :  14,
  'B' :  15,
  'C' :  16,
  'D' :  17,
  'E' :  18,
  'F' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'One morning, ' ---- characters mapped to int ---- > [27 49 40  2 48 50 53 49 44 49 42  8  2]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

#could not get following loop to work
#because tf.enable_eager_execution is not 
#viable on 2.0
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

O
n
e
 
m


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'One morning, when Gregor Samsa woke from troubled dreams, he found\r\nhimself transformed in his bed in'
'to a horrible vermin.  He lay on\r\nhis armour-like back, and if he lifted his head a little he could\r\n'
'see his brown belly, slightly domed and divided by arches into stiff\r\nsections.  The bedding was hard'
'ly able to cover it and seemed ready\r\nto slide off any moment.  His many legs, pitifully thin compare'
'd\r\nwith the size of the rest of him, waved about helplessly as he\r\nlooked.\r\n\r\n"What\'s happened to me?'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'One morning, when Gregor Samsa woke from troubled dreams, he found\r\nhimself transformed in his bed i'
Target data: 'ne morning, when Gregor Samsa woke from troubled dreams, he found\r\nhimself transformed in his bed in'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 27 ('O')
  expected output: 49 ('n')
Step    1
  input: 49 ('n')
  expected output: 40 ('e')
Step    2
  input: 40 ('e')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 48 ('m')
Step    4
  input: 48 ('m')
  expected output: 50 ('o')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 62) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           15872     
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_3 (Dense)              (64, None, 62)            63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([ 2, 23, 61, 26, 49, 48,  2, 17, 27, 14, 27, 21,  8, 57, 43, 42, 24,
       18, 56,  4, 56, 44, 31, 27, 47, 30,  6,  8, 50, 57, 30, 43, 56, 29,
       19, 55, 35, 41, 21, 59, 34, 31, 51, 21, 27, 13, 52, 53,  5, 10, 11,
       61, 19, 24,  1, 18, 16, 38, 43, 23, 47, 56, 10, 34, 23, 38, 33, 43,
       37, 16, 18, 58, 38, 45, 52, 40, 61, 30, 58, 28, 46, 56, 30, 11, 55,
       24,  3, 10,  9,  6, 34, 47,  2, 58, 30, 48, 32, 16, 31, 37])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'were.  Now and then he stood up from the table and took\r\nsome receipt or document from the little ca'

Next Char Predictions: 
 ' JzNnm DOAOH,vhgLEu"uiTOlS(,ovShuQFtYfHxWTpHO?qr\'.:zFL\rECchJlu.WJcVhbCEwcjqezSwPkuS:tL!.-(Wl wSmUCTb'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 62)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1260643


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
!mkdir training_checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=100

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
18/18 [==============================] - 5s 298ms/step - loss: 2.9577
Epoch 2/100
18/18 [==============================] - 5s 289ms/step - loss: 2.7216
Epoch 3/100
18/18 [==============================] - 5s 293ms/step - loss: 2.4060
Epoch 4/100
18/18 [==============================] - 5s 289ms/step - loss: 2.2595
Epoch 5/100
18/18 [==============================] - 5s 291ms/step - loss: 2.1626
Epoch 6/100
18/18 [==============================] - 5s 291ms/step - loss: 2.0659
Epoch 7/100
18/18 [==============================] - 5s 292ms/step - loss: 1.9818
Epoch 8/100
18/18 [==============================] - 5s 296ms/step - loss: 1.9022
Epoch 9/100
18/18 [==============================] - 5s 291ms/step - loss: 1.8265
Epoch 10/100
18/18 [==============================] - 5s 287ms/step - loss: 1.7556
Epoch 11/100
18/18 [==============================] - 5s 287ms/step - loss: 1.6909
Epoch 12/100
18/18 [==============================] - 5s 287ms/step - loss: 1.6271
Epoch 13/100


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_4 (Dense)              (1, None, 62)             63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.75

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"So it goes "))

So it goes that they had more ffrcousted, and she was clearly with a litely she
position and the forch had only been able to speak to his sister and thank her
for all that she had to do for him to ket himself as he looked round in the
darkness.  He soon made the discovery that he had ne
ligged the burnith pefforct the whole even in ond comples lay then st all about the houriously soon.

The cis of to way of stubberned f om had and firthen besome side a time and and espeain te
offece as he falled - and carried e prasen and werk as where they caught for his mother's
screams.  "Anna! Ann't
rowant.  He trought to he penturely with a grand announcement of it before the
next with it housed for a wairto clere him with lotther and looked around in omening heard as if they had
looked out of the beds flow under the sheet; he gave up the chance to see his
mother until later and was simply glad that see what Gregor came back from his business
tripss, morning in mean mare from his mother was coul
